In [90]:
import random

random.seed(42)

# Словарь fallback-значений для нетерминалов, чтобы на максимальной глубине ветка завершалась корректно.

terminal_generators = {
    "BINOP_FUNC": lambda: random.choice(["\\min", "\\max"]),
    "FUNCTION": lambda: random.choice(
        [
            "\\sin",
            "\\cos",
            "\\tan",
            "\\log",
            "\\ln",
            "\\exp",
            "\\sqrt",
            "\\arcsin",
            "\\arccos",
            "\\arctan",
        ]
    ),
    "FRAC": lambda: "\\frac",
    "NUMBER": lambda: (
        str(random.randint(1, 9))
        if random.random() < 0.8
        else str(random.randint(0, 9999)) + "." + str(random.randint(0, 9999))
    ),
    "GREEK": lambda: random.choice(
        [
            "\\alpha",
            "\\beta",
            "\\gamma",
            "\\delta",
            "\\epsilon",
            "\\zeta",
            "\\eta",
            "\\phi",
            "\\kappa",
            "\\lambda",
            "\\mu",
            "\\nu",
            "\\xi",
            "\\pi",
            "\\rho",
            "\\sigma",
            "\\tau",
            "\\varepsilon",
            "\\phi",
            "\\varphi",
            "\\chi",
            "\\psi",
            "\\omega",
        ]
    ),
    "LATIN": lambda: "".join(
        random.choices("abcdefghijklmnopqrstuvwxyz", k=random.randint(1, 4))
    ),
    "CAPS_LATIN": lambda: "".join(
        random.choices("abcdefghijklmnopqrstuvwxyz".upper(), k=random.randint(1, 4))
    ),
    "INTEGRAL": lambda: "\\int",
    "SUMMARY": lambda: "\\sum",
    "PROD": lambda: "\\prod",
    "LIMIT": lambda: "\\lim",
}


def get_terminal():
    return terminal_generators[
        random.choice(
            [
                "BINOP_FUNC",
                "FUNCTION",
                "FRAC",
                "NUMBER",
                "GREEK",
                "LATIN",
                "CAPS_LATIN",
                "INTEGRAL",
                "LIMIT",
                "SUMMARY",
                "PROD",
            ]
        )
    ]()


fallback_dict = {
    "expr": "-" + get_terminal() if random.random() < 0.5 else get_terminal(),
    "sum": get_terminal() + "+"+ get_terminal(),
    "product": get_terminal(),
    "power": get_terminal(),
    "postfix": get_terminal(),
    "primary": get_terminal(),
    "group": rf"{{{get_terminal()}}}",
    "frac_expr": rf"{{{get_terminal()}}}{{{get_terminal()}}}",
    "integral_limits": get_terminal(),
    "expr_opt": get_terminal(),
    "limit_limits": f"{{{terminal_generators['LATIN']()}}} = {{{get_terminal()}}}",
}

In [97]:
def generate_formula(
    grammar,
    weights,
    terminal_generators,
    symbol="start",
    depth=0,
    max_depth=10,
    recursion_bonus=0.5,
    default_weight=0.8,
):
    """
    Рекурсивная генерация формулы с фиксированной максимальной глубиной.
    Если достигнута максимальная глубина, для нетерминалов возвращаются
    заранее заданные fallback-значения, чтобы ветки завершались терминалами.

    :param grammar: словарь грамматики, где ключ – нетерминал, а значение – список альтернатив (каждая альтернатива – список токенов)
    :param weights: словарь весов для операторов и спец-токенов
    :param terminal_generators: словарь генераторов для терминальных символов
    :param symbol: текущий символ (нетерминал или терминал)
    :param depth: текущая глубина рекурсии
    :param max_depth: максимальная допустимая глубина рекурсии
    :param recursion_bonus: вес для рекурсивного вызова (если текущий символ встречается внутри своей же альтернативы)
    :param default_weight: вес по умолчанию для токенов, отсутствующих в weights
    :return: сгенерированная строка (формула)
    """
    # Если глубина превышает max_depth – возвращаем пустую строку.
    if depth > max_depth:
        return "", {}
    
    # Если мы на максимальной глубине и symbol – нетерминал, возвращаем fallback.
    if depth == max_depth and symbol in grammar:
        token = fallback_dict.get(symbol, "")
        return token, {token: 1}
    
    # Если symbol не является ключом грамматики, значит это терминал.
    if symbol not in grammar:
        if symbol in terminal_generators:
            token = terminal_generators[symbol]()
        else:
            token = symbol
        return token, {token: 1}

    alternatives = grammar[symbol]
    alt_weights = []
    for alt in alternatives:
        total_weight = 0
        for token in alt:
            if token == symbol:
                effective = recursion_bonus if depth < max_depth else default_weight
            else:
                effective = weights.get(token, default_weight)
            total_weight += effective
        alt_weights.append(total_weight)
    
    # Инвертированные веса: чем меньше сумма весов, тем выше вероятность выбора
    epsilon = 1e-6
    inv_weights = [1 / (w + epsilon) for w in alt_weights]
    total_inv = sum(inv_weights)
    probabilities = [w / total_inv for w in inv_weights]
    
    chosen_alt = random.choices(alternatives, weights=probabilities, k=1)[0]
    total_counts = {}
    result = ""
    for token in chosen_alt:
        # Если токен – нетерминал, то если следующая глубина равна max_depth, используем fallback,
        # иначе продолжаем рекурсию.
        if token in grammar:
            if depth + 1 == max_depth:
                result += fallback_dict.get(token, "")
            else:
                sub_formula, sub_counts = generate_formula(
                grammar,
                weights,
                terminal_generators,
                token,
                depth + 1,
                max_depth,
                recursion_bonus,
                default_weight,
                )
                result += sub_formula
                for k, v in sub_counts.items():
                    total_counts[k] = total_counts.get(k, 0) + v
        else:
            if token in terminal_generators:
                result += terminal_generators[token]()
            else:
                result += token
        
            
    return result, total_counts

In [98]:
grammar = {
    "start": [["expr"]],
    "expr": [["sum"]],
    "sum": [
        ["product"],
        ["(", "sum", ")", "product"],
        ["(", "sum", ")", "-", "(", "product", ")"],
    ],
    "product": [
        ["power"],
        ["product", "\\cdot", " ", "power"],
        ["product", "\\times", " ", "power"],
        ["product", "/", " ", "power"],
        ["product", "BINOP_FUNC", " ", "power"],
    ],
    "power": [
        ["{", "postfix", "}"],
        ["{", "postfix", "}", "^", "{", "power", "}"],
    ],
    "postfix": [
        ["{", "primary", "}"],
        ["{", "postfix", "}", "_", "{", "primary", "}"],
    ],
    "primary": [
        ["NUMBER"],
        ["GREEK"],
        ["LATIN"],
        ["CAPS_LATIN"],
        ["FRAC", "frac_expr"],
        ["BINOP_FUNC", " ", "group"],
        ["FUNCTION", " ", "group"],
        ["(", "expr", ")"],
        ["[", "expr", "]"],
        ["{", "expr", "}"],
        ["INTEGRAL", "integral_limits", " ", "expr_opt"],
        ["SUMMARY", "integral_limits", " ","expr_opt"],
        ["PROD", "integral_limits", " ", "expr_opt"],
        ["LIMIT", "limit_limits", " ","expr"],
        ["|", "expr", "|"],
    ],
    "group": [["{", "expr", "}"]],
    "frac_expr": [["{", "expr", "}", "{", "expr", "}"]],
    "integral_limits": [
        [" "],
        ["_", "group"],
        ["^", "group"],
        ["_", "group", "^", "group"],
    ],
    "expr_opt": [
        [" "],
        ["expr"],
    ],
    "limit_limits": [
        [" "],
        ["_", "group"],
    ],
}

weights = {
    "+": 1.0,
    "-": 1.0,
    "\\cdot": 1.0,
    "/": 1.0,
    "BINOP_FUNC": 1.0,
    "^": 1.0,
    "_": 1.0,
    "LIMIT": 1.0,
    "INTEGRAL": 1.0,
    "FUNCTION": 1.0,
    "FRAC": 1.0,
    "SUMMARY": 1.0,
    "PROD": 1.0,
}

In [99]:
# Генерация нескольких формул с фиксированной глубиной
n = 1000
formulas = [generate_formula(grammar, weights, terminal_generators) for _ in range(n)]

for formula in formulas:
    print("".join(formula[0]))



{{\sum\min HFT}}\times {{\prod  \max}}\max {{\pi}}\min {{\sum_{2+\prod}  }}
{{{\rho}}_{\lim_{2+\prod} \arctan}}^{{{{ozh}}_{|\lim|}}^{{{[2+\prod]}}}}
({{\tan {\lim}}}\cdot {{{ajdx}}_{edoz}}/ {{{{{\max}}}_{TIC}}_{|(2+\prod)-(\lim)|}})-({A}^{\arctan}/ {{1}}\max {{{1}}_{cga}}\max {{4951.6689}}^{{{1902.1592}}}\times {{rty}}/ {{{V}}_{\max {\lim}}})
{{7450.1501}}\min {{4}}\cdot {{{\int_{\lim} \max}}_{{(2+\prod)\lim}}}^{{{zo}}^{{{\sum\min HFT}}^{{{1}}}}}\times {{3}}^{{{[(2+\prod)\lim]}}^{{{S}}}}
{{{8}}_{3}}^{{{\max {\max}}}}\times {{CSWQ}}
{{SIIG}}
{{[\arctan]}}
{{{\prod   }}_{p}}^{{{\delta}}}
{{\frac{\lim}{\lim}}}
{{{\epsilon}}_{\frac{\lim}{(2+\prod)\lim}}}^{{{iiy}}}
{{KFNI}}/ {{\frac{\lim}{\lim}}}
(({{4}}){{1}}^{{A}}/ {{{1}}_{\frac{9}{LH}}}^{{{1}}^{{A}}}/ {{{2+\prod}}}\cdot {{{\int  \max}}_{WTFU}}^{{{XJ}}}){{\frac{9}{LH}}}/ {{\chi}}^{{{{A}_{1}}_{WQD}}}/ {{pjb}}\times {{bww}}
{{nylv}}^{{{A}_{1}}^{{A}}}\min {{{{1}}_{\max {\lim}}}_{kur}}/ {{crh}}\cdot {{\prod^{2+\prod}  }}^{{{{{\frac{9}{LH}}}_{

In [96]:
from IPython.display import display, Math

display(Math(formulas[2][0]))

<IPython.core.display.Math object>

In [100]:
display(Math(r"\frac{\,\,}{}"))

<IPython.core.display.Math object>